In [1]:
string = b"iVBORw0KGgoAAAANSUhEUgAAASwAAACoCAMAAABt9SM9AAAAh1BMVEX///8AAAD7+/vk5OT09PT39/fu7u6Ojo75+fnDw8Pc3NygoKDKyso1NTW6urrh4eGurq5vb2+AgIDU1NQgICCNjY0/Pz+ZmZl4eHiGhoZTU1OVlZVoaGi+vr61tbUqKipiYmKmpqYSEhInJyczMzNbW1tGRkZMTEwZGRlRUVEoKCgWFhYLCwvVI3uKAAASRUlEQVR4nOVdiZaiOhA1gCiKiqK4K9pu/dr//76XqkpCULRRoiP0PWemFRGSS6VSW2Kt9jK4o8NwOPwv8qz2fBof4pfdyOL/vDm8cpx8LbPdGx+Iv+oy9RsnmobPCLOah3+Pr7pR/8D/W7B2DW40hSP28BSFyQm9QbSO+oFkYBnx1nSW+HrAxnTwCGSP2ATfrJj49pzJz1+LnuCqY/EWAYIX3ajJGKehA/2c8Nv4/NCY/92rE0LREtFvOOkwZMRLR9Drs4j/vxZH+2yefPMNsmUNqYEn/ppe2S+6k8NYi/dvzak48NvENegsY2v5OSeTNdyav2MgfLUuf9tFuR/wVn7xdyBjAfuyas0fQfaBGKztX/mQNUyIoa2rZOxVd3KpgzFnbAd3bPIxxf+em+LzhuivteAaoVb7Em/5CAtRlvDMgJ0soJ2PBP5ZRAJmcx3C2OJVDVdoCcnv8ddtfBW/6la8Sx7/s6kBEcc9FzPrDDeUZDEcYQDXwSe3l18bAL2LDhsBWZxkTvvkCy4WsX6NGs5VyNerGq6wJq5QT85eqrKg+0vxcsjVEr+RyxZTxup0jItLg17hgRUOQgCqiIjNW6CWOFkWkDVfsR0MQxQn4Cx8nf6QGGsa1aHXL9OTtuo+f0Tj2nnA9ZHXRa0PaBFZzf4P245xUIrO70AzHPiHfS5rOAxdOLXD2e6gZPELr4DTFystwQ/J+xJfv27k24lk7fiI+mJcelxPkdWj+c4WenOgyIrg7ZF/g3PktdmQ66wznwW55vdpUmzjzPFCm4cw0YVpoJQXb3IYzAf9Y+e/9fq/zmwy9vMZkXehkdXhvESsPWLQZXlp0tm1nncGdtrqbMa+QUVM8dgJRh8nayy0BldnoEqC1fDVxgM9RZyoa0J98fbawWLLrnAc9wrezSEFD4h418Fs2OlkjVFAqFUhiBEZFRts4B5pAUa4ZVvfAlnY+qnqBROW2KtAgiVmEbrneXG4JkogmhdSoQ6T9jbv+hRI4INII8s6se3G6y7gVh7aWbHfWwKn/IwYyWoLnTFkMKeO+Lu50ruMJZd6AVy6g3jcq5skaZgt71/yHurJOJlwDQRW3QYmsbo8QfoS4yEe8+RNgeIFTkKgY0Gl72iI7nDK4Dpt3HLx/M3zrfsNpKSkcTPNQ1ahCeDALPGqDZ3dA3ku6CAJFyzxXVjzfXwbgmJYT+vUOqTH/0Gl0Sfl78IVHOUxMf1ahiGmQjmdd66J2X59fV0ds+5e9B7slnzVRDZW0OOwpZ/ier30N6QsuqKZzRDu7wgBt+E64VI0yV15tVchwM4P5dsUJfvRMrSpDe48zdYL9cIHg4z3kXyr2IgaWtSk5vVPKa4m/6Cl/x7CK1RiL8gY6MOi3l6nxSoOM65Ufdiko07qAEzLfU/XSK1JmqnD5i8OwabtSx40Oy5MS42fNri+gz/IlOVNDpoW8m+cFqaoitpvCnF/FMg+1pBNQjjTzxkVdXTKiNa13bnOOq+Xoqr9vGH1FD7hyTSDLKdvlnGmTunAMzf92awTRbP+WFmOvUEUTVsXZ01Vm/w1nens3s3fOIMpxkRyREc3iTicVkaboKIDW7LGN2KQp89K9OhQRCGm7L32yibF0Cke+7VvfHkZXOxF6qwv04FHIGvjb8Byg96jqTI46bkw0VIVeyf3Yqs8sndgo81+p31Abkwf317ITkOj1HgLbRHL3GNShNvEX3x2sfpp8caHhV4yGs3gNb8lySXbuFP9X7dbSluTx5eKuLhpX9q0Y2NTeAX+nix4WKSufnR3M+TMHMn4o5RrDch6YeQljbbq/DFlU9HI9K+OaDjfMsKegy1N4AhiWMqFX+ny3WeHWpdt4SWFs8ZA1lsy81xYjrLn+wstSTGH5KCzVyQlAnat/wtAkTUBWRJihocbqrlAnAg/+1wTtOlMo824CVU9cLyaUFZpyVJnQgbdGcg3fYONoWxpDYUdKBBhRFfTSQGOzS2qeB90F4TeNTJfiUB0eZcx93opnaViVn0K8iruInMeoSJrA1nog6wTCJM8BhycuDANOngcAqls8R6yJAPtrA+JDtIWTTUE1TP2VIjUWJ5XkTWADM1KinWUKIO6Eu8NzoYuJSXeMQxlYCHb/m0l5LgyaLXWT+2yu99/HEJnuW3q/YGdOUk2PCd5hkpRQKaQyMJyiMynbRSL++OIIvAg4GrIXSioujRQDckWJ2s/j/GKYMDU+SPq4GNSSoKbeRPP97t94KlHJAKBrwutCwgdvb/1OYl8nCi2DGtmYpQt6e6cFxTqsMhfnyZPsy/UggUmsStsi8HrKz6ELR7fPIGK2DqJ05hlVAmf+mwkmmVH/dG8HYQJOd5gvtIFfyoNqmWfj/2NkLjphfdoHPPfuJIFRtLBGWY/PcFWlPlhRbD8ZQwC+kzDLc0m7X+T9taHwc0jD3rQ5kaxjpfkdt7mnr0dFOYb1u+e5CdcZafke0eNzjfHlN4HoWh+sY8cxUNmeMG9yIO9vmLzn0DYTb/Wu0h/eZD1YZtd4qXFT/8M69viksLmtjbq6in7hZgKLqPlVQDJRI7iGxqHGeZVKmc4caUz0DHf1n8NJ78YcE9icG0yOLpNMSDzi4S1aHTXh0nCnVOw1ho7UEgUzBuNRrDizQ1GDUAQQNPd7rjR2HRJ7dZFMtxq8ymrvphyy7ZLlUcefn/UXvEPnCdydpPbeugaGZfXLYpYWqqkBk/XZz+ENfgvI9bBWdpnY60alLs3Sfq3pRxbtoUWbNhalMa3k/JAtrO0r/ehDPDhrIG4wLP98c5JA/badDq5qd7yw8Ew1UwkKwPQqv0dRoK+OkeXjORdpxPBg4bD6GXDopUJRiHALdmL53Y67KfdmjPb0STV6YQQfX7YLaJePZnwa30nVEUpZeYUewbyGiGlbyDWEYhkal/ORX32EyrLkDNq1Syoo1tB8S2pgBE7o+6Y2U11UdDQlvhKzuF00annVHESTM6ge15ctKgOHuM+IYTLyMHoygjMhJ2VteNIMcEBthfPaQT/L9MRm5YyqR8nizTOU/Ef3bK6jrbVCzwFgR7KR8S2MQRffDFhe3IywjER9ecwC3Bez3BshebdjJ0xw9MAsiCiFI9GozYpCVcZSeuHhyHGXQ5P9MTXcoYZU6SM+RRxenoomRF3RGHpji+KUhRZSq4DCuPuXWdES/tmbLIEfafI0nRCQtbk0YBq90nBcuOEqll2sMaRs87TsFE8DiwGgnhT/8Ojl2QNB26ysI+G1p7N+fQ4ATfOUmRt57JZok3tR58kWpPnR3thaVn7zs3EKk0dBZI9NiqiI4tBVry2kKylRpbSWRA2Oe+hRAWiTHtOcpOLZQfIWqZ1U1MF7VoPts29oXDuo5uYC6c7dmcox8izoOzOjMXQscNO6KxVQpaqMkcN6dLzcWE2bMg8J0qi3gZLlU01H5T6+RMPX/dt7tvAyGmBBRYukrVgcY1ooNlQkTXVyJJGUwelbYGydCSywotZ7+mVc9HD3XE132bQvH9uI1GrT8HBbo5Qx1iq1KgrY0lgU9k9f4nUCStlg8U8A9ThNt0dRk9j6aulF4os97EKYfdh9a4tnvj+NYUSFpwPm6jgu1TmoTKuanVmstDKBjWFutPHYbghMevqWwQo7SoVvPXg7EMTRX53cjlMuMrjV+HpBVLp7RNITUAN7C1o9DmRGFV1mRbfOzAmScynYBzbQzp12QFtJZO/Z9LNwVAUU846Dz1H9D5z0+tpyipfTQ9eP6sK1Tzsu2Fe537APBfQKc3nFrqjH01Z5RzsKLkFVoV9EuwHdMoooWqfu5aBoiPVWD7g5zYc/CRv88hAb+I3Cqxl/SC0c07tmrLKVbxdV/yjafL6kpZ3YJCLLH35xDyP/hmc2Uz4QKjhK7HgkAbJL0EUW1u7M8mlfcgUi/E1JiTv1gSUAW5vJTKrd2d2R1tpMsuX2ZKpWKRoA69KXSTS2uy1WNQ9Z0cL7w3z2vmq/k9tHvC6Fe4vhr25XBJ3W6Po4b384WEZPorhDZJ1KqOhZbXGO3aFW0l2Xa9PH+iuTDehhYHGybl8K1rDwdWOC3fIcrQqj/ixokOKNZATRa50yazS1jRjZ507ZGmJ0+jhJSZg7ItUQFg6yXLbV6OvM1kuxdEMsoIkEnp+KqGoVkf5JdNZvf4FUVHDo3FBk/xV7sxPVhCyxi/hvd+ACr40hVrORYXZYqmNiZ9EuSjoej0urGzQjiiL6ZCqMDv1L2wlDJ6l7Gs9aHwwUOeI1nw5jNKWvtXJ0b9SHYPLrmg7Ym2fX+7sz1UUFbl//XagBqAV2LBJlq+Ck/yPeusmCwjzecxZ6DVg3pVM4xXftFKyGJIhNc+evMnYFm8sLRfRf1ZZeWJyiMU1USu+b8FyAcg48OSWnUMmI4WngyQXEV1sMfP91cnXX0/No8I7ouxRKWq8qd2L2/Z30heNqsuY/AYP5qg83iQaUmYxPe1pfDhwSrsbL8fuTXv6jh+XyxrF2ovfSk5tvVhSed0NnbnPBvs1SHm1f8p1eE9OkHdjw2GsXyORJByWj1aM/Rv8Pg9d7KJ2zBAfaandXkjbGmsmP5tqdJNUvnP3jucBTzW8Na85vWWqk4ytM8N7YrnYjW2z7FXamWqk5hKSynLEHKj9wyieb4KuH7oOouV3g9Fsd2YXuBHeE0u2v68/aYaN9IYhX5c1NWhllcTZuaTjHgY3pyyc0fYph7pn+93B8eIK+yvZI55LYZI+Qtb98J6v75vq2OH1kiZ2mmcMNjrv1cuVDSE3V3kTe/5kd8r6/iJboWG8rBxeNB9A+6yeZSGfIzjL/G68vDGCqTigPOloNxh8Z/aQRYtBwHV+PURFn0uvbDJEanXHViUjrhTmu4JVb3mrzbQf72ff+328GAVd3Z5A9/mcJyAa6DTxOXZ83+cjwapE6l6B4jh5RKsu/cfZfGn/zi4ZcSVR73lBSdc8CRhr1WgHfk4b89U/BvJvQAVnGUZnIVg0bVZu4S85LIZ3JGlUUrBUEMZoiE4spamYxgJQKMpo2InCGe/Zee/N2JkeiGIdlLkLfhDEoDG2b6bp630WpkZVTP1UQXtUA1mQX2bKXciHHBYsk/hciBnRyBYfQmGVI0/xFMR2T7Pi1UEqzVNBu0FCxBSORdlKUmIlCZE+hZGRkailIEsSfH8Oop83Nj7MBScVmK+cY6hDZrWeXpikNns9VNeAV5C7Sjy59lSushu2ahW24BVk3ub8hHCpyjmou6TpohTlM89D5p/zLtRRsNTiTSzCIbutHFUOz6On6uUXj6Tdl+prgmSMvxbda+zzkRQvZJZWZsFTafyj9HFI11fUldaglnXxwejlsFFXmb9g0ZHqq+JwtbKt7eC+lm6Nkhx16rcc0JLPlWIrO9I/fTVZZquvetjQti9Yp1ktU21WQaQSqgzW+mx810nGZNMJN+nigMnlgEVj/pldzMqHmGVgGM328WIR7w+X9SEZZThUyVbh0EMC75qqmxhm/komVWdVc3/lC4idULPXc+o4TW7ZB7jEqvqmVk3q502t1c4uNCIMvDtFD3/F1KqJPX2op/4oi7B1v/GLB4mFJNVLSmcAUxhJ/bLrBY34+3g4HGf7fmPl5THuKUBWpjW/zwKdu2Jb7lD60Owvj34m0C4tWOP49IaxZQOapQVjLBQeK8eigUJAmzIudo38VYUlByYSiyZdDyYuUgKQVVqw1vjyl0mrCtr0oaBr9zeiy9K1K/o7gRSZqHxUy0ITvuha8OUfMbWORmYy9jdMrcd2x72Fyd8wtbCbhYusKPTwlp91/pdAf6d4JQxmeYa/n1duYEbsp/BEFvyJqJZvRttYeJnSb1X6C8igLP772YM/oeINuSrkN1X3F1oJWxNWaU3EXLfFr/PRmBmKr3T/gjc9NeQEW7gMu7LrLQhjU57KyES058NBTrCBC1HAtBT7sz2NlimySPuVZTeM52AbM5C86qt452ysi0hWtXPTaxOxUsQzv99WMpj7aRO3+jmx2Fy2YV/5QM3cnKahEEY1fq0oGxiKMrSVyNpI3PWDgU7d0MzO91T2UOFFdfpuwkVRr3q61TEpDX1j/sBnom7SpwurHgNErWxq1xVM81S4ogatI1O/Elr1ihrUM2tDF6PfvIsNXe3zgFbp1lTUjtI8lV2fsjTaPQr5VHZ9SmgodShwqLT1QMECY6l3SvNUtljLbPDc2lbaQcSIlrl6oUalVfzcrENHw7qqu640+1/Dk8EKGHcUsfNHujz/AxlRzUFY6syTAAAAAElFTkSuQmCC"

In [2]:
import base64
with open("imageToSave.png", "wb") as fh:
    fh.write(base64.decodebytes(string))